<a href="https://colab.research.google.com/github/joaochenriques/WE_LAB_2023/blob/main/DataCorrection/OWC_CamerasDataCorrection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import h5py
import matplotlib.pyplot as mpl
import numpy as np
import pandas as pd
import scipy.signal
from dataclasses import dataclass, field
import os, pathlib
from scipy import interpolate
import glob

In [ ]:
import subprocess

def cmdcall( cmd ):
    output = subprocess.getoutput( cmd )
    print(output)

In [ ]:
if not pathlib.Path("mpl_utils.py").exists():
  cmdcall( "curl -O https://raw.githubusercontent.com/joaochenriques/ipynb_libs/main/mpl_utils.py" ) 

import mpl_utils as mut
mut.config_plots()

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')

mpl.rcParams["figure.figsize"] = (6,4.5)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3921  100  3921    0     0  12567      0 --:--:-- --:--:-- --:--:-- 12607


In [ ]:
if not pathlib.Path("YB2m_Tests_V6.h5").exists():
    cmdcall( "curl -O https://raw.githubusercontent.com/joaochenriques/WE_LAB_2023/main/YB2m_Tests_V6.zip" )
    cmdcall( "unzip YB2m_Tests_V6.zip" )

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 4604k    0  7747    0     0  18185      0  0:04:19 --:--:--  0:04:19 18142
100 4604k  100 4604k    0     0  9512k      0 --:--:-- --:--:-- --:--:-- 9512k
Archive:  YB2m_Tests_V6.zip
  inflating: YB2m_Tests_V6.h5        


In [ ]:
if not pathlib.Path("YB2m_Cameras_V6.zip").exists():
    cmdcall( "curl -O https://raw.githubusercontent.com/joaochenriques/WE_LAB_2023/main/YB2m_Cameras_V6.zip" )
    cmdcall( "unzip YB2m_Cameras_V6.zip" )

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0 10.0M    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
100 10.0M  100 10.0M    0     0  3040k      0  0:00:03  0:00:03 --:--:-- 3040k
Archive:  YB2m_Cameras_V6.zip
  inflating: YB2m_H0.02m_f0.64Hz.csv  
  inflating: YB2m_H0.02m_f0.70Hz.csv  
  inflating: YB2m_H0.02m_f0.74Hz.csv  
  inflating: YB2m_H0.02m_f0.76Hz.csv  
  inflating: YB2m_H0.02m_f0.80Hz.csv  
  inflating: YB2m_H0.02m_f0.84Hz.csv  
  inflating: YB2m_H0.02m_f0.88Hz.csv  
  inflating: YB2m_H0.02m_f0.90Hz.csv  
  inflating: YB2m_H0.02m_f0.92Hz.csv  
  inflating: YB2m_H0.02m_f0.96Hz.csv  
  inflating: YB2m_H0.02m_f

In [ ]:
if not pathlib.Path("KB4m_Tests_V6.h5").exists():
    cmdcall( "curl -O https://raw.githubusercontent.com/joaochenriques/WE_LAB_2023/main/KB4m_Tests_V6.zip" )
    cmdcall( "unzip KB4m_Tests_V6.zip" )

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2189k  100 2189k    0     0  2716k      0 --:--:-- --:--:-- --:--:-- 2713k
Archive:  KB4m_Tests_V6.zip
  inflating: KB4m_Tests_V6.h5        


In [ ]:
if not pathlib.Path("KB4m_Cameras_V6.zip").exists():
    cmdcall( "curl -O https://raw.githubusercontent.com/joaochenriques/WE_LAB_2023/main/KB4m_Cameras_V6.zip" )
    cmdcall( "unzip KB4m_Cameras_V6.zip" )

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 7056k    0  7747    0     0  12142      0  0:09:55 --:--:--  0:09:55 12142
100 7056k  100 7056k    0     0  9484k      0 --:--:-- --:--:-- --:--:-- 9471k
Archive:  KB4m_Cameras_V6.zip
  inflating: KB4m_H0.04m_f0.64Hz.csv  
  inflating: KB4m_H0.04m_f0.70Hz.csv  
  inflating: KB4m_H0.04m_f0.74Hz.csv  
  inflating: KB4m_H0.04m_f0.78Hz.csv  
  inflating: KB4m_H0.04m_f0.82Hz.csv  
  inflating: KB4m_H0.04m_f0.86Hz.csv  
  inflating: KB4m_H0.04m_f0.90Hz.csv  
  inflating: KB4m_H0.04m_f0.94Hz.csv  
  inflating: KB4m_H0.04m_f1.00Hz.csv  
  inflating: KB4m_H0.04m_f1.02Hz.csv  
  inflating: KB4m_H0.04m_f1.04Hz.csv  
  inflating: KB4m_H0.04m_f1.06Hz.csv  
  inflating: KB4m_H0.04m_f1.08Hz.csv  
  inflating: KB4m_H0.04m_f1.10Hz.csv  
  inflating: KB4m_H0.04m_f1.

In [ ]:
def read_hdf_array( hdf5_Input, group, name ):
    return np.array( hdf5_Input[ group + '/' + name ] )

def read_hdf_scalar( hdf5_Input, group, name ):
    return hdf5_Input[ group + '/' + name ][()]

def read_hdf_string( hdf5_Input, group, name ):
    return hdf5_Input[ group + '/' + name ][()]

def save_hdf_array( hdf5_Output, group, name, fdata ):
    hdf5_Output.create_dataset( group + '/' + name, data=fdata, 
                                compression="gzip", compression_opts=9  )

def save_hdf_scalar( hdf5_Output, group, name, fdata ):
    hdf5_Output.create_dataset( group + '/' + name, data=fdata )

def save_hdf_string( hdf5_Output, group, name, fdata ):
    hdf5_Output.create_dataset( group + '/' + name, data=fdata, 
                                dtype=save_hdf_string.dt_str )
save_hdf_string.dt_str = h5py.special_dtype( vlen=bytes )

In [ ]:
hf = h5py.File( 'YB2m_Tests_V5.h5', 'a' )
filenames = []
cases_names = list( hf.keys() )
for case_name in cases_names:
    var_entry = case_name + '/CamerasFileName'
    if var_entry in hf:
        filename = read_hdf_string( hf, case_name, 'CamerasFileName' ).decode()

        if os.path.isfile( './' + filename ):
            print( "Found: " + filename )
            filenames.append( filename )
        else:
            print( "NOT Found: " + filename )
            # del hf[ case_name + '/CamerasFileName' ]
            # save_hdf_string( hf, case_name, 'CamerasFileName', 'YB2m_H0.02m_f0.60Hz.csv' )

hf.close()

In [ ]:
def interpolate_nans(A):
    inds = np.arange( A.shape[0] )
    good = np.where( np.isfinite(A) )
    f = interpolate.interp1d( inds[good], A[good], bounds_error=False )
    B = np.where( np.isfinite(A), A, f(inds) )
    return B

In [ ]:
filenames = sorted( [ f for f in glob.glob("*.csv") ] )

for filename in filenames:
    df = pd.read_csv( filename, skiprows=(0,1,2,3,4,5,6), header=None )

    times = np.array( df[1] )

    roll  = np.array( df[2] )
    yaw   = np.array( df[3] )
    pitch = np.array( df[4] )
    surge = np.array( df[5] )
    heave = np.array( df[6] )
    sway  = np.array( df[7] )    

    if np.count_nonzero( np.isnan(roll) ) > 0:
        roll  = interpolate_nans( roll  )
        yaw   = interpolate_nans( yaw   )
        pitch = interpolate_nans( pitch )
        surge = interpolate_nans( surge )
        heave = interpolate_nans( heave )
        sway  = interpolate_nans( sway  )

    n = np.count_nonzero(~np.isnan(roll))
    print( filename + ' -- ' + str( len(roll) - n) )


    data = {   
        "time": times, 
        "surge" : surge, 
        "heave" : heave, 
        "sway"  : sway, 
        "roll"  : roll, 
        "yaw"   : yaw, 
        "pitch" : pitch
        }


KB4m_H0.04m_f0.64Hz.csv -- 0
KB4m_H0.04m_f0.70Hz.csv -- 0
KB4m_H0.04m_f0.74Hz.csv -- 0
KB4m_H0.04m_f0.78Hz.csv -- 0
KB4m_H0.04m_f0.82Hz.csv -- 0
KB4m_H0.04m_f0.86Hz.csv -- 0
KB4m_H0.04m_f0.90Hz.csv -- 0
KB4m_H0.04m_f0.94Hz.csv -- 0
KB4m_H0.04m_f1.00Hz.csv -- 0
KB4m_H0.04m_f1.02Hz.csv -- 0
KB4m_H0.04m_f1.04Hz.csv -- 0
KB4m_H0.04m_f1.06Hz.csv -- 0
KB4m_H0.04m_f1.08Hz.csv -- 0
KB4m_H0.04m_f1.10Hz.csv -- 0
KB4m_H0.04m_f1.12Hz.csv -- 0
KB4m_H0.04m_f1.14Hz.csv -- 0
KB4m_H0.04m_f1.16Hz.csv -- 0
KB4m_H0.04m_f1.18Hz.csv -- 0
KB4m_H0.04m_f1.22Hz.csv -- 0
KB4m_H0.04m_f1.28Hz.csv -- 0
KB4m_H0.04m_f1.34Hz.csv -- 0
KB4m_H0.04m_f1.42Hz.csv -- 0
KB4m_H0.04m_f1.50Hz.csv -- 0
YB2m_H0.02m_f0.64Hz.csv -- 0
YB2m_H0.02m_f0.70Hz.csv -- 0
YB2m_H0.02m_f0.74Hz.csv -- 0
YB2m_H0.02m_f0.76Hz.csv -- 0
YB2m_H0.02m_f0.80Hz.csv -- 0
YB2m_H0.02m_f0.84Hz.csv -- 0
YB2m_H0.02m_f0.88Hz.csv -- 0
YB2m_H0.02m_f0.90Hz.csv -- 0
YB2m_H0.02m_f0.92Hz.csv -- 0
YB2m_H0.02m_f0.96Hz.csv -- 0
YB2m_H0.02m_f1.00Hz.csv -- 0
YB2m_H0.02m_f1

In [ ]:
!zip YB2m_Cameras_V6.zip *.csv

  adding: YB2m_H0.02m_f0.64Hz.csv (deflated 67%)
  adding: YB2m_H0.02m_f0.70Hz.csv (deflated 66%)
  adding: YB2m_H0.02m_f0.74Hz.csv (deflated 66%)
  adding: YB2m_H0.02m_f0.76Hz.csv (deflated 65%)
  adding: YB2m_H0.02m_f0.80Hz.csv (deflated 65%)
  adding: YB2m_H0.02m_f0.84Hz.csv (deflated 66%)
  adding: YB2m_H0.02m_f0.88Hz.csv (deflated 65%)
  adding: YB2m_H0.02m_f0.90Hz.csv (deflated 65%)
  adding: YB2m_H0.02m_f0.92Hz.csv (deflated 64%)
  adding: YB2m_H0.02m_f0.96Hz.csv (deflated 67%)
  adding: YB2m_H0.02m_f1.00Hz.csv (deflated 65%)
  adding: YB2m_H0.02m_f1.02Hz.csv (deflated 64%)
  adding: YB2m_H0.02m_f1.04Hz.csv (deflated 64%)
  adding: YB2m_H0.02m_f1.06Hz.csv (deflated 64%)
  adding: YB2m_H0.02m_f1.08Hz.csv (deflated 63%)
  adding: YB2m_H0.02m_f1.10Hz.csv (deflated 64%)
  adding: YB2m_H0.02m_f1.12Hz.csv (deflated 64%)
  adding: YB2m_H0.02m_f1.14Hz.csv (deflated 64%)
  adding: YB2m_H0.02m_f1.16Hz.csv (deflated 63%)
  adding: YB2m_H0.02m_f1.18Hz.csv (deflated 64%)
  adding: YB2m_H0.02

In [ ]:
hf = h5py.File( 'YB4m_Tests_V5.h5', 'a' )
filenames = []
cases_names = list( hf.keys() )
for case_name in cases_names:
    var_entry = case_name + '/CamerasFileName'
    if var_entry in hf:
        filename = read_hdf_string( hf, case_name, 'CamerasFileName' ).decode()

        if os.path.isfile( './' + filename ):
            print( "Found: " + filename )
            filenames.append( filename )
        else:
            print( "NOT Found: " + filename )
            # del hf[ case_name + '/CamerasFileName' ]
            # save_hdf_string( hf, case_name, 'CamerasFileName', 'YB2m_H0.02m_f0.60Hz.csv' )

hf.close()

Found: YB4m_H0.04m_f0.30Hz.csv
Found: YB4m_H0.04m_f0.50Hz.csv
Found: YB4m_H0.04m_f0.60Hz.csv
Found: YB4m_H0.04m_f0.64Hz.csv
Found: YB4m_H0.04m_f0.70Hz.csv
Found: YB4m_H0.04m_f0.74Hz.csv
Found: YB4m_H0.04m_f0.76Hz.csv
Found: YB4m_H0.04m_f0.80Hz.csv
Found: YB4m_H0.04m_f0.84Hz.csv
Found: YB4m_H0.04m_f0.88Hz.csv
Found: YB4m_H0.04m_f0.90Hz.csv
Found: YB4m_H0.04m_f0.92Hz.csv
Found: YB4m_H0.04m_f0.96Hz.csv
Found: YB4m_H0.04m_f1.00Hz.csv
Found: YB4m_H0.04m_f1.02Hz.csv
Found: YB4m_H0.04m_f1.06Hz.csv
Found: YB4m_H0.04m_f1.08Hz.csv
Found: YB4m_H0.04m_f1.10Hz.csv
Found: YB4m_H0.04m_f1.12Hz.csv
Found: YB4m_H0.04m_f1.14Hz.csv
Found: YB4m_H0.04m_f1.16Hz.csv
Found: YB4m_H0.04m_f1.18Hz.csv
Found: YB4m_H0.04m_f1.20Hz.csv
Found: YB4m_H0.04m_f1.24Hz.csv
Found: YB4m_H0.04m_f1.28Hz.csv
Found: YB4m_H0.04m_f1.30Hz.csv
Found: YB4m_H0.04m_f1.34Hz.csv
Found: YB4m_H0.04m_f1.40Hz.csv
Found: YB4m_H0.04m_f1.50Hz.csv


In [ ]:
for filename in filenames:
    df = pd.read_csv( filename, skiprows=(0,1,2,3,4,5,6), header=None )
    df.keys()
    valid = True
    data = np.array(df[2])
    n = np.count_nonzero(~np.isnan(data))
    print( filename + ' -- ' + str(len(data)) + ' -- ' + str(n) )

YB4m_H0.04m_f0.30Hz.csv -- 15638 -- 1331
YB4m_H0.04m_f0.50Hz.csv -- 15117 -- 2508
YB4m_H0.04m_f0.60Hz.csv -- 24437 -- 4458
YB4m_H0.04m_f0.64Hz.csv -- 16644 -- 16644
YB4m_H0.04m_f0.70Hz.csv -- 15157 -- 15132
YB4m_H0.04m_f0.74Hz.csv -- 15928 -- 13903
YB4m_H0.04m_f0.76Hz.csv -- 15001 -- 15000
YB4m_H0.04m_f0.80Hz.csv -- 15695 -- 1134
YB4m_H0.04m_f0.84Hz.csv -- 17376 -- 12681
YB4m_H0.04m_f0.88Hz.csv -- 14741 -- 14684
YB4m_H0.04m_f0.90Hz.csv -- 15395 -- 8504
YB4m_H0.04m_f0.92Hz.csv -- 14869 -- 14869
YB4m_H0.04m_f0.96Hz.csv -- 15124 -- 15046
YB4m_H0.04m_f1.00Hz.csv -- 13944 -- 10785
YB4m_H0.04m_f1.02Hz.csv -- 15107 -- 14778
YB4m_H0.04m_f1.06Hz.csv -- 15285 -- 15284
YB4m_H0.04m_f1.08Hz.csv -- 15237 -- 14341
YB4m_H0.04m_f1.10Hz.csv -- 15365 -- 623
YB4m_H0.04m_f1.12Hz.csv -- 15016 -- 12107
YB4m_H0.04m_f1.14Hz.csv -- 15692 -- 15688
YB4m_H0.04m_f1.16Hz.csv -- 15742 -- 15470
YB4m_H0.04m_f1.18Hz.csv -- 15032 -- 14552
YB4m_H0.04m_f1.20Hz.csv -- 16076 -- 10494
YB4m_H0.04m_f1.24Hz.csv -- 15315 -- 14747